In [10]:
import os
import psycopg
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../src/.env")



True

In [11]:
conn = psycopg.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

print("Connexion PostgreSQL OK")


Connexion PostgreSQL OK


In [13]:
def create_conversation_list(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
        return [l.strip() for l in lines if l.strip() != ""]
        
corpus_list = create_conversation_list("../data/corpus.txt")
len(corpus_list), corpus_list[:5]


(1982,
 ['<01> hotesse',
  'h: U B S bonjour',
  '<02> client',
  "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e",
  '<03> hotesse+client'])

In [18]:
conn.rollback()


In [19]:
with conn.cursor() as cur:
    cur.execute("DROP TABLE IF EXISTS embeddings;")
    cur.execute("""
        CREATE TABLE embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT,
            embedding FLOAT8[]
        );
    """)
conn.commit()
print("Table embeddings créée SANS pgvector, 100% compatible Windows")


Table embeddings créée SANS pgvector, 100% compatible Windows


In [20]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [23]:
pip install sentence-transformers


  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ------------------ --------------------- 262.1/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/199.4 MB 1.9 MB/s eta 0:01:47
   ---------------------------------------- 1.0/199.4 MB 1.9 MB/s eta 0:01:47
   -----

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


c:\Users\Lenovo\Chatbot-Rag\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [25]:
def get_embedding(text):
    emb = model.encode(text)
    return emb.tolist() 


In [26]:
with conn.cursor() as cur:
    cur.execute("DROP TABLE IF EXISTS embeddings;")
    cur.execute("""
        CREATE TABLE embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT,
            embedding FLOAT8[]
        );
    """)
conn.commit()
print("Table embeddings créée ✔")


Table embeddings créée ✔


In [27]:
with conn.cursor() as cur:
    for text in corpus_list:
        emb = get_embedding(text)
        cur.execute(
            "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
            (text, emb)
        )
    conn.commit()

print("Tout est inséré gratuitement ✔")


Tout est inséré gratuitement ✔


In [28]:
import numpy as np

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [29]:
def similar_corpus(input_text, top_k=3):
    query_emb = get_embedding(input_text)

    with conn.cursor() as cur:
        cur.execute("SELECT id, corpus, embedding FROM embeddings")
        rows = cur.fetchall()

    results = []
    for id_, corpus, emb in rows:
        score = cosine_similarity(query_emb, emb)
        results.append((id_, corpus, score))

    # Trier par similarité descendante
    results.sort(key=lambda x: x[2], reverse=True)

    return results[:top_k]


In [30]:
def build_context(similar_results):
    return "\n".join([f"- {corpus}" for _, corpus, _ in similar_results])


In [33]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


In [42]:
import ollama

def generate_answer(question):
    similar = similar_corpus(question, top_k=3)
    context = build_context(similar)

    prompt = f"""
Tu es un assistant UBS.

Contexte :
{context}

Question :
{question}

Réponse (basée uniquement sur le contexte) :
"""
    response = ollama.generate(
    model="phi3:mini",
    prompt=prompt
)



    return response["response"]


In [43]:
generate_answer("Comment ouvrir un dossier à l'UBS ?")


"Pour accéder au dossier du programme de formation en droit économique et social, vous devrez suivre les étapes ci-dessous: Si je suis un assistant UBS spécialisé dans la gestion des dossiers institutionnels, j'informerais que l'ouverture d'un tel dossier nécessite une demande officielle. Vous pourriez envoyer votre candidature au service administratif de notre établissement via le formulaire fourni sur notre site internet ou directement par la poste accompagnée des informations requises telles que vos références, motivations et expériences pertinentes. Une fois votre demande reçue et évaluée conformément aux critères d'admission du programme, un responsable compétent déclenchera le processus de création de votre dossier si vous êtes sélectionné pour y participer.\n\nSi vous souhaitez obtenir des informations plus spécifiques ou aider avec une autre tâche liée à l'UBS, n'hésitez pas à me le faire savoir !"